In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import requests
import json
import datetime
import time

# 미세먼지 데이터 처리

In [2]:
#  온도/기온/풍속/습도/기압/일조/일사
file_list = os.listdir('./data/origin_data_file/weather')

pd_weather = []
for file_name in file_list:
    if file_name == '.ipynb_checkpoints':
        continue
    tmp = pd.read_csv(f'./data/origin_data_file/weather/{file_name}',encoding='cp949').drop(columns=['지점'])
    tmp['일시'] = pd.to_datetime(tmp['일시'])
    pd_weather.append(tmp)
pd_weather = pd.concat(pd_weather,axis=0).sort_values(by=['일시','지점명'])
pd_weather = pd_weather[pd_weather['지점명'] == '서울'].reset_index(drop=True)
pd_weather 

,지점명,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m)
0,서울,2016-01-01 00:00:00,-1.9,NaN,0.1,0.0,85.0,1022.1,NaN,NaN,0.0,0.0,NaN,500.0
1,서울,2016-01-01 01:00:00,-2.1,NaN,1.6,90.0,83.0,1022.0,NaN,NaN,NaN,NaN,NaN,600.0
2,서울,2016-01-01 02:00:00,-2.2,NaN,0.4,0.0,86.0,1022.2,NaN,NaN,NaN,NaN,NaN,500.0
3,서울,2016-01-01 03:00:00,-2.5,NaN,1.8,90.0,90.0,1022.2,NaN,NaN,0.0,0.0,NaN,400.0
4,서울,2016-01-01 04:00:00,-2.9,NaN,1.9,70.0,90.0,1022.0,NaN,NaN,0.0,0.0,NaN,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52603,서울,2021-12-31 19:00:00,-6.4,NaN,2.2,250.0,34.0,1021.1,NaN,NaN,0.0,0.0,NaN,2000.0
52604,서울,2021-12-31 20:00:00,-6.3,NaN,4.1,320.0,35.0,1021.2,NaN,NaN,0.0,0.0,NaN,2000.0
52605,서울,2021-12-31 21:00:00,-6.7,NaN,4.8,320.0,36.0,1021.2,NaN,NaN,0.0,0.0,NaN,2000.0
52606,서울,2021-12-31 22:00:00,-7.5,NaN,3.0,320.0,37.0,1021.8,NaN,NaN,0.0,0.0,NaN,2000.0


In [3]:
#  미세먼지
file_list = os.listdir('./data/origin_data_file/pm10')

pd_pm10 = []
for file_name in file_list:
    if file_name == '.ipynb_checkpoints':
        continue
    tmp = pd.read_csv(f'./data/origin_data_file/pm10/{file_name}',encoding='cp949').drop(columns=['지점'])
    tmp['일시'] = pd.to_datetime(tmp['일시'])
    pd_pm10.append(tmp)
pd_pm10 = pd.concat(pd_pm10,axis=0).sort_values(by=['일시','지점명']).reset_index(drop=True)
pd_pm10

,지점명,일시,1시간평균 미세먼지농도(㎍/㎥)
0,관악산,2016-01-01 00:00:00,26
1,서울,2016-01-01 00:00:00,73
2,수원,2016-01-01 00:00:00,70
3,관악산,2016-01-01 01:00:00,36
4,서울,2016-01-01 01:00:00,72
...,...,...,...
144083,서울,2021-12-31 22:00:00,26
144084,수원,2021-12-31 22:00:00,33
144085,관악산,2021-12-31 23:00:00,17
144086,서울,2021-12-31 23:00:00,24


In [4]:
pd_pm10_pivot = pd_pm10.pivot(index='일시',columns='지점명',values='1시간평균 미세먼지농도(㎍/㎥)').copy()
pd_pm10_pivot

지점명,관악산,서울,수원
일시,,,
2016-01-01 00:00:00,26.0,73.0,70.0
2016-01-01 01:00:00,36.0,72.0,71.0
2016-01-01 02:00:00,44.0,78.0,84.0
2016-01-01 04:00:00,53.0,89.0,103.0
2016-01-01 05:00:00,49.0,85.0,96.0
...,...,...,...
2021-12-31 19:00:00,35.0,22.0,28.0
2021-12-31 20:00:00,27.0,27.0,34.0
2021-12-31 21:00:00,37.0,23.0,25.0


In [5]:
pd_pm10.isna().sum()

지점명                  0
일시                   0
1시간평균 미세먼지농도(㎍/㎥)    0
dtype: int64

In [6]:
pd_weather['강수량(mm)'] = pd_weather['강수량(mm)'].fillna(value=0.0)
pd_weather['일조(hr)'] = pd_weather['일조(hr)'].fillna(value=0.0)
pd_weather['일사(MJ/m2)'] = pd_weather['일사(MJ/m2)'].fillna(value=0.0)
pd_weather['시정(10m)'] = pd_weather['시정(10m)'].interpolate()
pd_weather['기온(°C)'] = pd_weather['기온(°C)'].interpolate()
pd_weather['습도(%)'] = pd_weather['습도(%)'].interpolate()
pd_weather['풍속(m/s)'] = pd_weather['풍속(m/s)'].interpolate()
pd_weather['풍향(16방위)'] = pd_weather['풍향(16방위)'].fillna(method='ffill')
pd_weather['현지기압(hPa)'] = pd_weather['현지기압(hPa)'].interpolate()
pd_weather.index = pd_weather['일시']
pd_weather.drop(columns=['일시','지점명','전운량(10분위)','중하층운량(10분위)','최저운고(100m )'],inplace=True)
pd_weather

,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),일조(hr),일사(MJ/m2),시정(10m)
일시,,,,,,,,,
2016-01-01 00:00:00,-1.9,0.0,0.1,0.0,85.0,1022.1,0.0,0.0,500.0
2016-01-01 01:00:00,-2.1,0.0,1.6,90.0,83.0,1022.0,0.0,0.0,600.0
2016-01-01 02:00:00,-2.2,0.0,0.4,0.0,86.0,1022.2,0.0,0.0,500.0
2016-01-01 03:00:00,-2.5,0.0,1.8,90.0,90.0,1022.2,0.0,0.0,400.0
2016-01-01 04:00:00,-2.9,0.0,1.9,70.0,90.0,1022.0,0.0,0.0,400.0
...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,-6.4,0.0,2.2,250.0,34.0,1021.1,0.0,0.0,2000.0
2021-12-31 20:00:00,-6.3,0.0,4.1,320.0,35.0,1021.2,0.0,0.0,2000.0
2021-12-31 21:00:00,-6.7,0.0,4.8,320.0,36.0,1021.2,0.0,0.0,2000.0


In [7]:
pd_weather.isna().sum()

기온(°C)       0
강수량(mm)      0
풍속(m/s)      0
풍향(16방위)     0
습도(%)        0
현지기압(hPa)    0
일조(hr)       0
일사(MJ/m2)    0
시정(10m)      0
dtype: int64

In [8]:
total_wheather = pd.concat([pd_weather,pd_pm10_pivot],axis=1)
total_wheather['관악산'].interpolate(inplace=True)
total_wheather['서울'].interpolate(inplace=True)
total_wheather['수원'].interpolate(inplace=True)
total_wheather.columns = ['기온','강수량','풍속','풍향','습도','기압','일조량','일사량','시정','관악pm10','서울pm10','수원pm10']
total_wheather.to_csv('./data/weather_and_pm10.csv')

In [9]:
#길이 체크
6*365*24+24*2#2016, 2020 -> 윤년

52608

# 코스피 데이터 처리

In [12]:
import FinanceDataReader as fdr
import pandas_datareader as pdr

In [13]:
stocks = fdr.StockListing('KOSPI') # 코스피, 코스닥, 코넥스 전체
stocks

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,60200,1,800,1.35,59700,60300,59400,7144469,428879682300,359380909510000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,587000,2,-5000,-0.84,591000,596000,581000,162380,95422280000,137358000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,86200,1,1700,2.01,85400,86500,85000,1633646,140375186000,62753803863000,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,875000,1,4000,0.46,879000,879000,870000,19075,16685015000,62277250000000,71174000,STK
4,006400,KR7006400006,삼성SDI,KOSPI,,739000,1,6000,0.82,742000,749000,733000,161915,119946641000,50816987670000,68764530,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,001527,KR7001522002,동양2우B,KOSPI,,13250,1,350,2.71,13400,13400,13200,109,1444600,4088539250,308569,STK
937,002785,KR7002781003,진흥기업우B,KOSPI,,4750,2,-50,-1.04,4860,4860,4740,444,2104810,4071785500,857218,STK
938,001525,KR7001521004,동양우,KOSPI,,6400,1,50,0.79,6350,6400,6350,12,76250,3953478400,617731,STK
939,002787,KR7002782001,진흥기업2우B,KOSPI,,13300,1,200,1.53,13500,13500,13300,113,1521500,3920946400,294808,STK


In [14]:
KOSPI_VOL150_LIST = stocks.sort_values(by='Volume',ascending=False).head(200).copy().reset_index(drop=True)
KOSPI_VOL150_LIST

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,014990,KR7014990006,인디에프,KOSPI,,1550,1,315,25.51,1565,1605,1385,24298953,36241131400,109605142250,70712995,STK
1,001440,KR7001440007,대한전선,KOSPI,,1755,1,15,0.86,1785,1800,1755,16124940,28611956555,2184050130795,1244473009,STK
2,096300,KR7096300009,베트남개발1,KOSPI,,211,2,-5,-2.31,218,220,209,11592857,2460276480,29050524521,137680211,STK
3,005930,KR7005930003,삼성전자,KOSPI,,60200,1,800,1.35,59700,60300,59400,7144469,428879682300,359380909510000,5969782550,STK
4,034020,KR7034020008,두산에너빌리티,KOSPI,,15450,1,550,3.69,15100,15600,14850,6729003,102827458850,9861859109850,638308033,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,004920,KR7004920005,씨아이테크,KOSPI,,870,1,12,1.40,872,925,820,176075,154992823,37940754810,43610063,STK
196,011690,KR7011690005,와이투솔루션,KOSPI,,618,1,5,0.82,612,622,607,175649,108019204,98407623666,159235637,STK
197,033780,KR7033780008,KT&G,KOSPI,,94400,2,-1800,-1.87,96200,96200,94200,168651,16015177800,12960411716800,137292497,STK
198,011170,KR7011170008,롯데케미칼,KOSPI,,170500,1,13000,8.25,158500,171000,158500,167658,27894099500,5843958939500,34275419,STK


In [15]:
start_date = '2016-01-01'
end_date = '2022-01-01'

stock_data = []
col_name = []
for i in range(200):
    Code = KOSPI_VOL150_LIST['Code'][i]
    Name = KOSPI_VOL150_LIST['Name'][i]
    
    tmp = fdr.DataReader(Code,start=start_date,end=end_date).Close
    
    if tmp.shape[0] < 0.85*(52*5*6):#(52주 * 5영업일 * 6년)
        print(f'{Name:<15} : {tmp.shape[0]}')
        continue
    
    col_name.append(f'{Name}_close')
    stock_data.append(tmp)
stock_data = pd.concat(stock_data,axis=1)
stock_data.columns = col_name
stock_data

카카오뱅크           : 99
우리금융지주          : 715
한컴라이프케어         : 93
두산퓨얼셀           : 547
화승알앤에이          : 201
카카오페이           : 42
SK바이오사이언스       : 198
대덕전자            : 402
현대에너지솔루션        : 525
SK아이이테크놀로지      : 161
두산퓨얼셀1우         : 547
애경산업            : 932
화승엔터프라이즈        : 1292
한화시스템           : 529
현대건설기계          : 1144
에스디바이오센서        : 114
일동제약            : 1312
SK스퀘어           : 24
케이카             : 57
티웨이항공           : 843
에어부산            : 744


,인디에프_close,대한전선_close,베트남개발1_close,삼성전자_close,두산에너빌리티_close,이구산업_close,금양_close,유니온머티리얼_close,현대두산인프라코어_close,그린케미칼_close,...,금호석유_close,평화산업_close,대호에이엘_close,고려산업_close,GKL_close,씨아이테크_close,와이투솔루션_close,KT&G_close,롯데케미칼_close,대우부품_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,2740,2103,100,24100,14766,1420,1855,2725,3724,3433,...,52300,1631,1191,2010,23400,2091,1520,105000,245000,1310
2016-01-05,2965,2141,104,24160,14689,1500,1895,2670,3762,3371,...,52200,1635,1171,2020,23900,2048,1534,105500,250000,1275
2016-01-06,2930,2103,102,23500,14421,1510,1910,2760,3698,3300,...,53000,1626,1179,2045,23100,2048,1643,106500,260000,1290
2016-01-07,2900,2094,102,23260,14191,1455,1880,2590,3644,3333,...,51900,1559,1125,2010,22450,1983,1564,104000,253500,1260
2016-01-08,2730,2037,101,23420,13885,1445,1880,2595,3546,3333,...,52800,1559,1094,1975,22500,1876,1534,107000,264500,1260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,1255,1665,50,80500,19545,3590,4700,3115,6990,9300,...,184500,1820,3460,2900,13200,1545,1200,84200,228500,2525
2021-12-27,1275,1661,51,80200,19449,3575,4735,3130,6830,9260,...,186000,1805,3450,2935,13100,1455,1200,84200,229500,2480
2021-12-28,1250,1675,51,80300,20028,3590,4750,3230,6770,9240,...,185000,1710,3400,2900,13250,1455,1200,83900,229000,2420


In [16]:
stock_data.isna().sum()

인디에프_close       0
대한전선_close       0
베트남개발1_close     0
삼성전자_close       0
두산에너빌리티_close    0
                ..
씨아이테크_close      0
와이투솔루션_close     0
KT&G_close       0
롯데케미칼_close      0
대우부품_close       0
Length: 179, dtype: int64

In [17]:
stock_data = stock_data.fillna(method='ffill')
stock_data = stock_data.fillna(value=0.0)
stock_data

,인디에프_close,대한전선_close,베트남개발1_close,삼성전자_close,두산에너빌리티_close,이구산업_close,금양_close,유니온머티리얼_close,현대두산인프라코어_close,그린케미칼_close,...,금호석유_close,평화산업_close,대호에이엘_close,고려산업_close,GKL_close,씨아이테크_close,와이투솔루션_close,KT&G_close,롯데케미칼_close,대우부품_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,2740,2103,100,24100,14766,1420,1855,2725,3724,3433,...,52300,1631,1191,2010,23400,2091,1520,105000,245000,1310
2016-01-05,2965,2141,104,24160,14689,1500,1895,2670,3762,3371,...,52200,1635,1171,2020,23900,2048,1534,105500,250000,1275
2016-01-06,2930,2103,102,23500,14421,1510,1910,2760,3698,3300,...,53000,1626,1179,2045,23100,2048,1643,106500,260000,1290
2016-01-07,2900,2094,102,23260,14191,1455,1880,2590,3644,3333,...,51900,1559,1125,2010,22450,1983,1564,104000,253500,1260
2016-01-08,2730,2037,101,23420,13885,1445,1880,2595,3546,3333,...,52800,1559,1094,1975,22500,1876,1534,107000,264500,1260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,1255,1665,50,80500,19545,3590,4700,3115,6990,9300,...,184500,1820,3460,2900,13200,1545,1200,84200,228500,2525
2021-12-27,1275,1661,51,80200,19449,3575,4735,3130,6830,9260,...,186000,1805,3450,2935,13100,1455,1200,84200,229500,2480
2021-12-28,1250,1675,51,80300,20028,3590,4750,3230,6770,9240,...,185000,1710,3400,2900,13250,1455,1200,83900,229000,2420


주요 지수 | 코스피지수 | 코스닥지수 | 원-달러 환율 | 원-엔화 환율 | S&P500 | NASDAQ | 미 10년채 금리 |    국제 원유(두바이유 : 국내 유가 영향)
  ---     |    ---     |     ---    |     ---      |      ---     |   ---  |   ---  |     ---        |              ---
__코드__  |    KS11    |    KQ11    |    USD/KRW   |    JPY/KRW   |  US500 |  IXIC  |     DGS10      |           POILDUBUSDM


In [18]:
KOSPI = fdr.DataReader('KS11',start=start_date,end=end_date)['Close']
KOSPI

Date
2016-01-04    1918.760010
2016-01-05    1930.530029
2016-01-06    1925.430054
2016-01-07    1904.329956
2016-01-08    1917.619995
                 ...     
2021-12-24    3012.429932
2021-12-27    2999.550049
2021-12-28    3020.239990
2021-12-29    2993.290039
2021-12-30    2977.649902
Name: Close, Length: 1473, dtype: float64

In [19]:
LIST_1 = ['KQ11','USD/KRW','JPY/KRW','US500','IXIC']

pd_index1 = []
for L in LIST_1:
    tmp = fdr.DataReader(L,start=start_date,end=end_date)['Close']
    pd_index1.append(tmp)
pd_index1 = pd.concat(pd_index1,axis=1)
pd_index1.columns = LIST_1
pd_index1

,KQ11,USD/KRW,JPY/KRW,US500,IXIC
Date,,,,,
2016-01-01,NaN,1174.109985,9.580900,NaN,NaN
2016-01-04,677.789978,1174.109985,9.585800,2012.660034,4903.089844
2016-01-05,684.070007,1187.359985,9.946800,2016.709961,4891.430176
2016-01-06,687.270020,1188.719971,9.988000,1990.260010,4835.759766
2016-01-07,679.659973,1198.739990,10.111000,1943.089966,4689.430176
...,...,...,...,...,...
2021-12-27,1011.359985,1185.920044,10.363698,4791.189941,15871.259766
2021-12-28,1027.439941,1185.650024,10.334326,4786.350098,15781.719727
2021-12-29,1028.050049,1187.849976,10.349199,4793.060059,15766.219727


In [20]:
A = pdr.DataReader('DGS10','fred',start=start_date, end=end_date)
B = pdr.DataReader('POILDUBUSDM', 'fred', start=start_date, end=end_date)

pd_index2 = pd.concat([A,B],axis=1)
pd_index2 = pd_index2.fillna(method='ffill')
pd_index2

,DGS10,POILDUBUSDM
DATE,,
2016-01-01,NaN,27.458095
2016-01-04,2.24,27.458095
2016-01-05,2.25,27.458095
2016-01-06,2.18,27.458095
2016-01-07,2.16,27.458095
...,...,...
2021-12-28,1.49,72.847826
2021-12-29,1.55,72.847826
2021-12-30,1.52,72.847826


In [21]:
total_stock = pd.merge(left=KOSPI, right=pd_index1, how='left', left_index=True, right_index=True)
total_stock.rename(columns={'Close':"KOSPI"},inplace=True)
total_stock = pd.merge(left=total_stock, right=pd_index2, how='left', left_index=True, right_index=True)
total_stock = pd.merge(left=total_stock, right=stock_data, how='left', left_index=True, right_index=True)
total_stock.fillna(method='ffill',inplace=True)
total_stock

,KOSPI,KQ11,USD/KRW,JPY/KRW,US500,IXIC,DGS10,POILDUBUSDM,인디에프_close,대한전선_close,...,금호석유_close,평화산업_close,대호에이엘_close,고려산업_close,GKL_close,씨아이테크_close,와이투솔루션_close,KT&G_close,롯데케미칼_close,대우부품_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,1918.760010,677.789978,1174.109985,9.585800,2012.660034,4903.089844,2.24,27.458095,2740,2103,...,52300,1631,1191,2010,23400,2091,1520,105000,245000,1310
2016-01-05,1930.530029,684.070007,1187.359985,9.946800,2016.709961,4891.430176,2.25,27.458095,2965,2141,...,52200,1635,1171,2020,23900,2048,1534,105500,250000,1275
2016-01-06,1925.430054,687.270020,1188.719971,9.988000,1990.260010,4835.759766,2.18,27.458095,2930,2103,...,53000,1626,1179,2045,23100,2048,1643,106500,260000,1290
2016-01-07,1904.329956,679.659973,1198.739990,10.111000,1943.089966,4689.430176,2.16,27.458095,2900,2094,...,51900,1559,1125,2010,22450,1983,1564,104000,253500,1260
2016-01-08,1917.619995,682.559998,1195.390015,10.174000,1922.030029,4643.629883,2.13,27.458095,2730,2037,...,52800,1559,1094,1975,22500,1876,1534,107000,264500,1260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,3012.429932,1007.419983,1185.500000,10.357149,4725.790039,15653.370117,1.50,72.847826,1255,1665,...,184500,1820,3460,2900,13200,1545,1200,84200,228500,2525
2021-12-27,2999.550049,1011.359985,1185.920044,10.363698,4791.189941,15871.259766,1.48,72.847826,1275,1661,...,186000,1805,3450,2935,13100,1455,1200,84200,229500,2480
2021-12-28,3020.239990,1027.439941,1185.650024,10.334326,4786.350098,15781.719727,1.49,72.847826,1250,1675,...,185000,1710,3400,2900,13250,1455,1200,83900,229000,2420


In [22]:
total_stock.isna().sum().sum()

0

In [23]:
total_stock.to_csv('./data/kospi_financeIndex_and_stock.csv')